In [1]:
from selenium import webdriver
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time
from scrapy.http import TextResponse

In [2]:
%%time
title_ls = []
clicked_ls = []
time_ls = []
link_ls = []

for page in range(1,182+1):
    if page % 100 ==0 : print('{} page is over.'.format(page))
    url = 'http://cointalk.co.kr/bbs/board.php?bo_table=coinnews&page={}'.format(page)
    req = requests.get(url)
    response = TextResponse(req.url, body=req.text, encoding='utf-8')
    for content in range(1,35+1):
        if content % 10 == 0 : time.sleep(1)
        try : 
            title_ls.append(response.xpath('//*[@id="fboardlist"]/table/tbody/tr[{}]/td[2]/a/text()'.format(content)).extract()[0])
            click_process = ','.join(re.findall('\d',response.xpath('//*[@id="fboardlist"]/table/tbody/tr[{}]/td[4]/text()'.format(content)).extract()[0])).replace(',','')
            clicked_ls.append(click_process)
            time_ls.append(response.xpath('//*[@id="fboardlist"]/table/tbody/tr[{}]/td[7]/text()'.format(content)).extract()[0])
            link_ls.append(response.xpath('//*[@id="fboardlist"]/table/tbody/tr[{}]/td[2]/a/@href'.format(content)).extract()[0])
        except : print('pass')

100 page is over.
CPU times: user 59.4 s, sys: 104 ms, total: 59.5 s
Wall time: 11min 47s


In [3]:
df = pd.DataFrame()

df['title'] = title_ls
df['time'] = time_ls
df['click'] = clicked_ls
df['link'] = link_ls

In [4]:
df.tail()

,title,time,click,link
6365,"비트코인 이어 이더리움도 관심, '스마트 콘트랙트' 전자계약이 뭐길래?",05-25,3190,http://cointalk.co.kr/bbs/board.php?bo_table=c...
6366,비트코인보다 눈부신 이더리움…올해 들어 2300% 급등,05-25,2706,http://cointalk.co.kr/bbs/board.php?bo_table=c...
6367,비트코인 이더리움 가상화폐 수요 급증 이유는,05-25,3067,http://cointalk.co.kr/bbs/board.php?bo_table=c...
6368,몸값 오르는 가상화폐…비트코인·이더리움 뭐길래?,05-25,2801,http://cointalk.co.kr/bbs/board.php?bo_table=c...
6369,비트코인 2500달러 또 사상최고치,05-25,2341,http://cointalk.co.kr/bbs/board.php?bo_table=c...


In [19]:
req = requests.get(df['link'].values[20])
response = TextResponse(req.url, body=req.text, encoding='utf-8')
len(','.join(response.xpath('//*[@id="st-view"]/section[1]/article/p/text()').extract()))

1363

In [ ]:
//*[@id="st-view"]/section[1]/article/p[4]

In [ ]:
%%time
content_ls = []
for idx in range(len(df)):
    if idx % 10 ==0 : print(idx)
    req = requests.get(df['link'].values[idx])
    response = TextResponse(req.url, body=req.text, encoding='utf-8')
    testing_ls = ','.join(response.xpath('//*[@id="st-view"]/section[1]/article/p/span/text()').extract() + \
    response.xpath('//*[@id="st-view"]/section[1]/article/p/text()').extract()).replace('\xa0','').replace(',','')
    process_ls = testing_ls
    content_ls.append(process_ls)

0
10
20
30
40
50
60
70
80
90


In [ ]:
df1 = pd.DataFrame()

df1['title'] = title_ls
df1['time'] = time_ls
df1['click'] = clicked_ls
df1['content'] = content_ls

In [ ]:
df1.head()

In [ ]:
df1.tail()

In [ ]:
df1.to_csv('cointalk_data.csv',index=False)